In [ ]:
# Exercice 1 
db.getCollection('university').aggregate(
  [
    {
      $match: {
        state: {
          $in: ['California', 'Massachusetts']
        }
      }
    },
    {
      $lookup: {
        from: 'course',
        localField: '_id',
        foreignField: 'university',
        as: 'courses'
      }
    },
    {
      $unwind: {
        path: '$courses',
        includeArrayIndex: 'string',
        preserveNullAndEmptyArrays: true
      }
    },
    {
      $project: {
        _id: 0,
        university_name: '$name',
        kind: 1,
        city: 1,
        course_name: '$courses.name',
        cfu: '$courses.cfu',
        teacher_name:
          '$courses.teacher.name',
        teacher_surname:
          '$courses.teacher.surname',
        participants: '$courses.participants'
      }
    }
  ],
);

In [ ]:
# Exercice 2
[
    {
        '$addFields': {
            'participants_number': {
                '$size': '$participants'
            }
        }
    }, {
        '$sort': {
            'participants_number': -1
        }
    }, {
        '$group': {
            '_id': '$university', 
            'topCourses': {
                '$push': {
                    'course_name': '$name', 
                    'teacher_name': '$teacher.name', 
                    'teacher_surname': '$teacher.surname', 
                    'participants_number': '$participants_number'
                }
            }
        }
    }, {
        '$project': {
            'top3': {
                '$slice': [
                    '$topCourses', 3
                ]
            }
        }
    }, {
        '$lookup': {
            'from': 'university', 
            'localField': '_id', 
            'foreignField': '_id', 
            'as': 'university'
        }
    }, {
        '$unwind': {
            'path': '$university'
        }
    }, {
        '$project': {
            '_id': 0, 
            'university_name': '$university.name', 
            'teachers_of_top3_courses': {
                '$map': {
                    'input': '$top3', 
                    'as': 'course', 
                    'in': {
                        'course_name': '$$course.course_name', 
                        'teacher_name': '$$course.teacher_name', 
                        'teacher_surname': '$$course.teacher_surname', 
                        'participants_number': '$$course.participants_number'
                    }
                }
            }
        }
    }
]

In [ ]:
# Exercice 3
[
    {
        '$unwind': {
            'path': '$exams'
        }
    }, {
        '$lookup': {
            'from': 'course', 
            'localField': 'exams.course', 
            'foreignField': '_id', 
            'as': 'course'
        }
    }, {
        '$unwind': {
            'path': '$course'
        }
    }, {
        '$match': {
            'course.exam_type': {
                '$in': [
                    'Written', 'Both'
                ]
            }
        }
    }, {
        '$addFields': {
            'passed': {
                '$gte': [
                    '$exams.mark', 18
                ]
            }
        }
    }, {
        '$group': {
            '_id': '$_id', 
            'name': {
                '$first': '$name'
            }, 
            'surname': {
                '$first': '$surname'
            }, 
            'totalWrittenExams': {
                '$sum': 1
            }, 
            'passedWrittenExams': {
                '$sum': {
                    '$cond': [
                        '$passed', 1, 0
                    ]
                }
            }, 
            'avgMark': {
                '$avg': '$exams.mark'
            }
        }
    }, {
        '$match': {
            '$expr': {
                '$and': [
                    {
                        '$gt': [
                            '$totalWrittenExams', 0
                        ]
                    }, {
                        '$eq': [
                            '$passedWrittenExams', '$totalWrittenExams'
                        ]
                    }, {
                        '$gt': [
                            '$avgMark', 27
                        ]
                    }
                ]
            }
        }
    }
]

In [ ]:
# Exercice 4
[
    {
        '$project': {
            'name': 1, 
            'participants': 1
        }
    }, {
        '$lookup': {
            'from': 'student', 
            'localField': 'participants', 
            'foreignField': '_id', 
            'as': 'parti'
        }
    }, {
        '$unwind': {
            'path': '$parti'
        }
    }, {
        '$project': {
            'name': 1, 
            'parti': 1
        }
    }, {
        '$unwind': {
            'path': '$parti.exams'
        }
    }, {
        '$lookup': {
            'from': 'course', 
            'localField': 'parti.exams.course', 
            'foreignField': '_id', 
            'as': 'exam'
        }
    }, {
        '$project': {
            '_id': '$_id', 
            'course_name': '$name', 
            'fullName': {
                '$concat': [
                    '$parti.name', ' ', '$parti.surname'
                ]
            }, 
            'exam_mark': '$parti.exams.mark', 
            'exam_name': '$exam.name'
        }
    }, {
        '$unwind': {
            'path': '$exam_name'
        }
    }, {
        '$match': {
            '$expr': {
                '$eq': [
                    '$course_name', '$exam_name'
                ]
            }
        }
    }, {
        '$group': {
            '_id': '$_id', 
            'course_name': {
                '$first': '$course_name'
            }, 
            'student_info': {
                '$push': {
                    'fullname': '$fullName', 
                    'exam_mark': '$exam_mark'
                }
            }
        }
    }, {
        '$addFields': {
            'students_count': {
                '$size': {
                    '$ifNull': [
                        '$student_info', []
                    ]
                }
            }, 
            'mark18_23': {
                '$size': {
                    '$filter': {
                        'input': {
                            '$ifNull': [
                                '$student_info', []
                            ]
                        }, 
                        'as': 's', 
                        'cond': {
                            '$and': [
                                {
                                    '$gte': [
                                        '$$s.exam_mark', 18
                                    ]
                                }, {
                                    '$lte': [
                                        '$$s.exam_mark', 23
                                    ]
                                }
                            ]
                        }
                    }
                }
            }, 
            'mark24_27': {
                '$size': {
                    '$filter': {
                        'input': {
                            '$ifNull': [
                                '$student_info', []
                            ]
                        }, 
                        'as': 's', 
                        'cond': {
                            '$and': [
                                {
                                    '$gte': [
                                        '$$s.exam_mark', 24
                                    ]
                                }, {
                                    '$lte': [
                                        '$$s.exam_mark', 27
                                    ]
                                }
                            ]
                        }
                    }
                }
            }, 
            'markAbove27': {
                '$size': {
                    '$filter': {
                        'input': {
                            '$ifNull': [
                                '$student_info', []
                            ]
                        }, 
                        'as': 's', 
                        'cond': {
                            '$gt': [
                                '$$s.exam_mark', 27
                            ]
                        }
                    }
                }
            }, 
            'fail': {
                '$size': {
                    '$filter': {
                        'input': {
                            '$ifNull': [
                                '$student_info', []
                            ]
                        }, 
                        'as': 's', 
                        'cond': {
                            '$lt': [
                                '$$s.exam_mark', 18
                            ]
                        }
                    }
                }
            }
        }
    }
]

In [ ]:
# Exercice 5
[
    {
        '$project': {
            'name': 1, 
            'surname': 1, 
            'friends': 1
        }
    }, {
        '$lookup': {
            'from': 'student', 
            'localField': 'friends', 
            'foreignField': '_id', 
            'as': 'directFriends'
        }
    }, {
        '$match': {
            'directFriends': {
                '$ne': []
            }
        }
    }, {
        '$unwind': {
            'path': '$directFriends'
        }
    }, {
        '$project': {
            '_id': '$_id', 
            'name': '$name', 
            'surname': '$surname', 
            'f_name': '$directFriends.name', 
            'f_surname': '$directFriends.surname', 
            'fof': '$directFriends.friends'
        }
    }, {
        '$match': {
            'fof': {
                '$ne': []
            }
        }
    }, {
        '$lookup': {
            'from': 'student', 
            'localField': 'fof', 
            'foreignField': '_id', 
            'as': 'fof_info'
        }
    }, {
        '$unwind': {
            'path': '$fof_info'
        }
    }, {
        '$project': {
            '_id': '$_id', 
            'name': '$name', 
            'surname': '$surname', 
            'f_name': '$f_name', 
            'f_surname': '$f_surname', 
            'fof_name': '$fof_info.name', 
            'fof_surname': '$fof_info.surname'
        }
    }, {
        '$match': {
            '$expr': {
                '$ne': [
                    'fof_name', 'name'
                ]
            }
        }
    }, {
        '$project': {
            '_id': 0, 
            'name': '$name', 
            'surname': '$surname', 
            'fof_name': '$fof_name', 
            'fof_surname': '$fof_surname'
        }
    }, {
        '$group': {
            '_id': {
                'name': '$name', 
                'surname': '$surname', 
                'fof_name': '$fof_name', 
                'fof_surname': '$fof_surname'
            }
        }
    }, {
        '$project': {
            '_id': 0, 
            'name': '$_id.name', 
            'surname': '$_id.surname', 
            'fof_name': '$_id.fof_name', 
            'fof_surname': '$_id.fof_surname'
        }
    }
]